# PointNet++ для семантической сегментации облака точек

Этот ноутбук настраивает окружение и запускает обучение модели PointNet++


## 1. Установка зависимостей


In [ ]:
# Установка всех необходимых библиотек
%pip install torch torchvision numpy scikit-learn tqdm matplotlib tensorboard -q


## 2. Клонирование репозитория или загрузка файлов


In [ ]:
# Клонировать репозиторий (данные уже включены!)
!git clone https://github.com/JohnSili/pointnet2-segmentation.git
%cd pointnet2-segmentation


## 3. Распаковка данных

**Данные уже включены в репозиторий!** Архив `data_for_colab.zip` содержит 500 PLY файлов.

Если нужно загрузить свои данные, используйте вариант B ниже.


In [ ]:
import zipfile
import os

# Распаковать данные из репозитория
if os.path.exists('data_for_colab.zip'):
    print("Распаковка данных...")
    with zipfile.ZipFile('data_for_colab.zip', 'r') as zip_ref:
        zip_ref.extractall('.')
    print("✓ Данные распакованы!")
else:
    raise FileNotFoundError("Архив данных не найден! Убедитесь, что репозиторий клонирован полностью.")

# Проверка наличия данных
data_dir = '3011-20251217T195928Z-1-001'
area = '3011'
if not os.path.exists(os.path.join(data_dir, area)):
    raise FileNotFoundError(f"Данные не найдены в {data_dir}/{area}")

ply_files = [f for f in os.listdir(os.path.join(data_dir, area)) if f.endswith('.ply')]
print(f"✓ Найдено {len(ply_files)} PLY файлов")


## 4. Проверка данных


In [ ]:
from dataset import S3DISDataset
import torch
import os

# Используем реальные данные
data_dir = '3011-20251217T195928Z-1-001'
area = '3011'

# Проверка данных
print("Проверка данных...")
dataset = S3DISDataset(data_dir, area=area, split='train', num_points=2048)
points, labels = dataset[0]
unique_classes = sorted(torch.unique(labels).tolist())

print(f"✓ Датасет: {len(dataset)} файлов")
print(f"✓ Форма: {points.shape[0]} точек, {points.shape[1]} признаков")
print(f"✓ Классы: {unique_classes} ({len(unique_classes)} классов)")

# Сохраняем для использования в обучении
DATA_DIR = data_dir
AREA = area
NUM_CLASSES = len(unique_classes)


## 5. Обучение модели


In [ ]:
# Запуск обучения
!python train.py \
    --data_dir {DATA_DIR} \
    --area {AREA} \
    --num_points 2048 \
    --batch_size 8 \
    --epochs 50 \
    --lr 0.001 \
    --num_classes {NUM_CLASSES} \
    --device cuda \
    --save_dir ./checkpoints \
    --log_dir ./logs


## 6. Мониторинг обучения (TensorBoard)


In [ ]:
# Запуск TensorBoard
%load_ext tensorboard
%tensorboard --logdir ./logs


## 7. Тестирование модели


In [ ]:
!python test.py \
    --data_dir {DATA_DIR} \
    --area {AREA} \
    --checkpoint ./checkpoints/best_model.pth \
    --num_classes {NUM_CLASSES}


## 8. Визуализация результатов


In [ ]:
!python visualize.py \
    --data_dir {DATA_DIR} \
    --area {AREA} \
    --checkpoint ./checkpoints/best_model.pth \
    --num_samples 5 \
    --output_dir ./visualizations


## 9. Скачивание результатов


In [ ]:
from google.colab import files
import zipfile
import os

# Создаем архив с результатами
with zipfile.ZipFile('results.zip', 'w') as zipf:
    if os.path.exists('checkpoints'):
        for root, dirs, files_list in os.walk('checkpoints'):
            for file in files_list:
                zipf.write(os.path.join(root, file))
    if os.path.exists('visualizations'):
        for root, dirs, files_list in os.walk('visualizations'):
            for file in files_list:
                zipf.write(os.path.join(root, file))

# Скачиваем архив
files.download('results.zip')
